In [3]:
import itk

#file='series1_tilescan4_channel3_cropped_sp200_0_ep800_600' #'series1_tilescan4_channel3_cropped_sp300_300_ep600_600'
file='series1_tilescan4_channel3_cropped_sp300_300_ep600_600' #'series1_tilescan4_channel3_cropped_sp300_300_ep600_600'

In [13]:
# Median Filtering 
radius=4 #radius of kernel for median filter
dimension=3
InputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/'+file+'.tif'
OutputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/median.mha'
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New(FileName=InputFileName)

MedianFilter=itk.MedianImageFilter[ImageType,ImageType].New()
MedianFilter.SetInput(reader.GetOutput())
MedianFilter.SetRadius(radius)

writer=itk.ImageFileWriter[ImageType].New()
writer.SetFileName(OutputFileName)
writer.SetInput(MedianFilter.GetOutput())
writer.Update()
# PixelType=itk.UC
# ImageType=itk.Image[PixelType,dimension]
# RescaleFilter=itk.RescaleIntensityImageFilter[Image]

In [5]:
# Multiscale Hessian-Based Measure and Rescaling to [0 255] of Median Image
#Multiscale Hessian-Based Measure
SigmaMin=2
SigmaMax=5
NumSigmaSteps=5
InputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/median.mha'
OutputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/plane.mha'
InputImage=itk.imread(InputFileName,itk.F)
ImageType=type(InputImage)
dimension=3
HessianPixelType=itk.SymmetricSecondRankTensor[itk.D,dimension]
HessianImageType=itk.Image[HessianPixelType,dimension]

ObjectnessFilter=itk.HessianToObjectnessMeasureImageFilter[HessianImageType,ImageType].New()
ObjectnessFilter.SetBrightObject(True)
ObjectnessFilter.SetScaleObjectnessMeasure(False)
ObjectnessFilter.SetAlpha(1.0)
ObjectnessFilter.SetBeta(1.0)
ObjectnessFilter.SetGamma(1.0)
ObjectnessFilter.SetObjectDimension(2) #M=2 for 2D plate-like structures

MultiScaleFilter=itk.MultiScaleHessianBasedMeasureImageFilter[ImageType,HessianImageType, ImageType].New()
MultiScaleFilter.SetInput(InputImage)
MultiScaleFilter.SetHessianToMeasureFilter(ObjectnessFilter)
MultiScaleFilter.SetSigmaStepMethodToLogarithmic()
MultiScaleFilter.SetSigmaMaximum(SigmaMax)
MultiScaleFilter.SetSigmaMinimum(SigmaMin)
MultiScaleFilter.SetNumberOfSigmaSteps(NumSigmaSteps)

#Rescaling
OutputPixelType=itk.UC
OutputImageType=itk.Image[OutputPixelType,dimension]

RescaleFilter=itk.RescaleIntensityImageFilter[ImageType,OutputImageType].New()
RescaleFilter.SetInput(MultiScaleFilter)

itk.imwrite(RescaleFilter.GetOutput(),OutputFileName)


In [2]:
#Watershed of Median Image
InputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/median-flip.tif'
OutputFileName=\
'/home/jwk114/CardiomyocytesSegmentation/Images/'+file+'/watershed-flip.mha'
dimension=3
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New()
reader.SetFileName(InputFileName)

WatershedFilter=itk.WatershedImageFilter.New(Input=reader.GetOutput())
threshold=0
level=0.2
WatershedFilter.SetThreshold(threshold)
WatershedFilter.SetLevel(level)

LabelledImageType=type(WatershedFilter.GetOutput())
PixelType=itk.UC
RGBPixelType=itk.RGBPixel[PixelType]
RGBImageType=itk.Image[RGBPixelType,dimension]

ScalarToRGBColormapFilterType=\
itk.ScalarToRGBColormapImageFilter[LabelledImageType,RGBImageType]
ColormapImageFilter = ScalarToRGBColormapFilterType.New()
ColormapImageFilter.SetColormap(ScalarToRGBColormapFilterType.Jet)
ColormapImageFilter.SetInput(WatershedFilter.GetOutput())
ColormapImageFilter.Update()

WriterType = itk.ImageFileWriter[RGBImageType]
writer = WriterType.New()
writer.SetFileName(OutputFileName)
writer.SetInput(ColormapImageFilter.GetOutput())
writer.Update()


In [145]:
# Morphological Watershed (can't get it to work)
InputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/median.mha'
OutputFileName='/home/jwk114/CardiomyocytesSegmentation/Images/morphWatershed.tif'
dimension=3
PixelType=itk.F
ImageType=itk.Image[PixelType,dimension]
reader=itk.ImageFileReader[ImageType].New()
reader.SetFileName(InputFileName)

MorphWatershedFilter=itk.MorphologicalWatershedImageFilter.New(Input=reader.GetOutput())
level=0.5
threshold=0.005
MorphWatershedFilter.SetLevel(level)
MorphWatershedFilter.SetFullyConnected(True)
MorphWatershedFilter.Update()
#MorphWatershedFilter.SetThreshold(threshold)

LabelledImageType=type(MorphWatershedFilter.GetOutput())
PixelType=itk.UC
RGBPixelType=itk.RGBPixel[PixelType]
RGBImageType=itk.Image[RGBPixelType,dimension]

ScalarToRGBColormapFilterType=\
itk.ScalarToRGBColormapImageFilter[LabelledImageType,RGBImageType]
ColormapImageFilter = ScalarToRGBColormapFilterType.New()
ColormapImageFilter.SetColormap(ScalarToRGBColormapFilterType.Jet)
ColormapImageFilter.SetInput(MorphWatershedFilter.GetOutput())
ColormapImageFilter.Update()

WriterType = itk.ImageFileWriter[RGBImageType]
#WriterType = itk.ImageFileWriter[type(MorphWatershedFilter.GetOutput())]
writer = WriterType.New()
writer.SetFileName(OutputFileName)
writer.SetInput(ColormapImageFilter.GetOutput())
#writer.SetInput(MorphWatershedFilter.GetOutput())
writer.Update()





'choohoo'